In [1]:
import numpy as np
import pandas as pd
import import_ipynb
import glob

In [2]:
countries={ 'Austria': 'AT', 'Belgium': 'BE',  'Bulgaria': 'BG', 'Switzerland': 'CH', 'Czech Republic': 'CZ',  'Germany': 'DE', 'Denmark': 'DK', 'Estonia': 'EE', 'Spain': 'ES', 'Finland': 'FI', 'France': 'FR',  'Greece': 'GR', 'Hungary': 'HU', 'Ireland': 'IE', 'Italy': 'IT', 'Lithuania': 'LT', 'Latvia': 'LV', 'Montenegro': 'ME','Netherlands': 'NL', 'Norway': 'NO', 'Poland': 'PL', 'Portugal': 'PT', 'Serbia': 'RS', 'Sweden': 'SE', 'Slovenia': 'SI', 'Slovakia': 'SK', 'United Kingdom': 'UK'}

abbr_list=list(countries.values())

In [3]:
fuels=['Biomass', 'Fossil Brown coal/Lignite', 'Fossil Coal-derived gas', 'Fossil Gas', 'Fossil Hard coal', 'Fossil Oil', 'Fossil Oil shale', 'Fossil Peat', 'Geothermal', 'Hydro Pumped Storage', 'Hydro Pumped Storage', 'Hydro Run-of-river and poundage', 'Hydro Water Reservoir', 'Marine', 'Nuclear', 'Other', 'Other renewable', 'Solar', 'Waste', 'Wind Offshore', 'Wind Onshore']

In [4]:
import matplotlib.pyplot as plt 

# from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import train_test_split

In [18]:
import data_preperation as dp
import copy
import pyomo.environ as pyo
from pyomo.opt import SolverFactory


In [23]:
load_dic = dp.load(countries)[1]
generation_dic = dp.generation(countries)[1]
transmission_data = dp.cross_border(abbr_list)[1]
load_dic_copy = copy.deepcopy(load_dic)
generation_dic_copy = copy.deepcopy(generation_dic)
transmission_data_copy = copy.deepcopy(transmission_data)
for abbr in abbr_list:
    load_dic[abbr] = load_dic[abbr].fillna(0)
    gen_dic[abbr] = gen_dic[abbr].fillna(0)

transmission_data = transmission_data.fillna(0)


In [24]:
country_index = list(countries.values())
time_index = np.arange(8760, dtype=int)
generation_index = {}

for abbr, df in generation_dic.items():
    generation_index[abbr] = [x for x in df.columns.values]

generation_fuels = np.array(
    list(set([item for sublist in generation_index.values() for item in sublist])))
generation_fuels.sort()

imports_index = {}
exports_index = {}

for abbr in abbr_list:
    imports_index[abbr] = [
        x for x in transmission_data.columns.values if abbr in x[-2:]]
    exports_index[abbr] = [
        x for x in transmission_data.columns.values if abbr in x[:2]]

transmission_index = np.array(transmission_data.columns.values)
transmission_index.sort()


In [25]:
model = pyo.ConcreteModel()

model.country_index = pyo.Set(initialize=country_index)
model.time_index = pyo.Set(initialize=time_index)
model.generation_fuels = pyo.Set(initialize=generation_fuels)
model.transmission_index = pyo.Set(initialize=transmission_index)

model.alpha = pyo.Var(model.country_index, model.time_index,
                      model.generation_fuels, bounds=(1.0, None))
model.gamma = pyo.Var(model.country_index, model.time_index, bounds=(1.0, None))
model.beta = pyo.Var(model.time_index, model.transmission_index, bounds=(1.0, None))

print('VARIABLES DECLARED')

def balance_rule(model, country, time):
    return sum(model.alpha[country, time, generation] * generation_dic[country][generation][time] for generation in generation_index[country]) + sum(model.beta[time, link] * transmission_data[link][time] for link in imports_index[country]) ==  \
        model.gamma[country, time] * load_dic[country]["demand"][time] + sum(model.beta[time, link] * transmission_data[link][time]
                                        for link in exports_index[country])

def ObjRule(model):
    return sum(model.alpha[country, time, generation] ** 2 for country in model.country_index for time in model.time_index for generation in generation_index[country]) \
        + sum(model.beta[time, link] ** 2 for time in model.time_index for link in model.transmission_index)\
        + sum(model.gamma[country, time] **
              2 for country in model.country_index for time in model.time_index)
model.obj = pyo.Objective(rule=ObjRule, sense=pyo.minimize)
opt = SolverFactory("gurobi", solver_io="python")


opt.solve(model)
print('OPTIMIZATION COMPLETED')

In [ ]:
generation_index_copy = copy.deepcopy(generation_index)

intermediary_var = {}
intermediary_var["generation"] = {}
intermediary_var["load"] = {}
unit_var = {}
unit_var["generation"] = {}
unit_var["load"] = {}

for country in country_index:
    table_gen = []
    row_load = []
    for time in time_index:
        row_gen = []
        for generation in generation_index_copy[country]:
            row_gen.append(model.delta_generation[country, time, generation].value)
        table_gen.append(row_gen)
        row_load.append(model.delta_load[country, time].value)

    intermediary_var["generation"][country] = pd.DataFrame.from_records(table_gen)
    intermediary_var["load"][country] = pd.DataFrame(row_load)
    intermediary_var["generation"][country].columns = generation_index_copy[country]
    intermediary_var["load"][country].columns = ['demand']
    intermediary_var["generation"][country].to_csv("../Data Sources/output/Sigma/Debugging/" + "sigma_gen_" + country + ".csv")
    intermediary_var["load"][country].to_csv("../Data Sources/output/Sigma/Debugging/" + "sigma_load_" + country + ".csv")
    unit_var["generation"][country] = pd.DataFrame(1, index=np.arange(df.shape[0]), columns=generation_index_copy[country])
    unit_var["load"][country] = pd.DataFrame(1, index=np.arange(df.shape[0]), columns=['demand'])

table_transmission = []

for time in time_index:
    row = []
    for link in transmission_index:
        row.append(model.delta_transmission[time, link].value)
    table_transmission.append(row)
intermediary_var["transmission"] = pd.DataFrame.from_records(table_transmission)
intermediary_var["transmission"].columns = transmission_index
intermediary_var["transmission"].to_csv("../Data Sources/output/Sigma/Debugging/sigma_transmission.csv")
unit_var["transmission"] = pd.DataFrame(1, index=np.arange(transmission_data.shape[0]), columns=transmission_data.columns)

generation_dic_copy = copy.deepcopy(generation_dic)
load_dic_copy = copy.deepcopy(load_dic)
transmission_data_copy = copy.deepcopy(transmission_data)

# We send the intermediary_var and unit_var to 'data_preperation.ipynb' to get the consolidated generation,load and transmission values.

dp.data_consolidation('Sigma',generation_dic_copy, load_dic_copy, transmission_data_copy, intermediary_var, unit_var)

In [7]:

# INDEX WORK
n_index = np.asarray(list(g_d.keys()))
# Index for time steps
t_index = list(g_d.values())[0].index.values
# Production type indices
p_index = {}
for country, df in g_d.items():
    p_index[country] = [
        x for x in df.columns.values if "demand" not in x if "s_r" not in x]
# Index for all production types
p_global = np.array(
    list(set([item for sublist in p_index.values() for item in sublist])))
p_global.sort()
# Prepare the country-dependent indices
# Direction import
i_index = {}
for country in g_d.keys():
    i_index[country] = [x for x in F.columns.values if country in x[-2:]]
# Direction export
e_index = {}
for country in g_d.keys():
    e_index[country] = [x for x in F.columns.values if country in x[:2]]
# Index for all links
f_index = np.array(F.columns.values)
f_index.sort()
# PYOMO WORK
model = pyo.ConcreteModel()
# Index declartions
model.n_index = pyo.Set(initialize=n_index)
model.t_index = pyo.Set(initialize=t_index)
model.p_global = pyo.Set(initialize=p_global)
model.f_index = pyo.Set(initialize=f_index)
# Variable declarations
model.alpha = pyo.Var(model.n_index, model.t_index,
                        model.p_global, bounds=(1.0, None))
model.gamma = pyo.Var(model.n_index, model.t_index, bounds=(1.0, None))
model.omega = pyo.Var(model.n_index, model.t_index, bounds=(1.0, None))
model.beta = pyo.Var(model.t_index, model.f_index, bounds=(1.0, None))

def balance_rule(model, n, t):
    return sum(model.alpha[n, t, p] * g_d[n][p][t] for p in p_index[n]) + sum(model.beta[t, l] * F[l][t] for l in i_index[n]) ==  \
        model.gamma[n, t] * g_d[n]["demand"][t] + model.omega[n, t] * \
        g_d[n]["s_r"][t] + sum(model.beta[t, l] * F[l][t]
                                for l in e_index[n])
model.balance_rule = pyo.Constraint(
    model.n_index, model.t_index, rule=balance_rule)

def ObjRule(model):
    return sum(model.alpha[n, t, p] ** 2 for n in model.n_index for t in model.t_index for p in p_index[n]) \
        + sum(model.beta[t, l] ** 2 for t in model.t_index for l in model.f_index)\
        + sum(model.gamma[n, t] ** 2 for n in model.n_index for t in model.t_index)\
        + sum(model.omega[n, t] **
                2 for n in model.n_index for t in model.t_index)
model.obj = pyo.Objective(rule=ObjRule, sense=pyo.minimize)
opt = SolverFactory("gurobi", solver_io="python")
# Solve the model
opt.solve(model)
# Convert the pyomo variables to pandas data frames
# alpha, gamma
a_var = {}
a_var["g_d"] = {}
f_var = {}
f_var["g_d"] = {}
for n in n_index:
    table = []
    for t in t_index:
        row = []
        for p in p_index[n]:
            row.append(model.alpha[n, t, p].value)
        # Append gamma value to "row"
        row.append(model.gamma[n, t].value)
        row.append(model.omega[n, t].value)
        table.append(row)
    f_var["g_d"][n] = pd.DataFrame.from_records(table)
    p_index[n].append("demand")
    p_index[n].append("s_r")
    f_var["g_d"][n].columns = p_index[n]
    a_var["g_d"][n] = pd.DataFrame(
        0, index=np.arange(df.shape[0]), columns=g_d[n].columns)
    #if not os.path.exists("../output/debugging/factorial_approach/"):
    #os.makedirs("../output/debugging/factorial_approach/")
    #f_var["g_d"][n].to_csv("../output/debugging/factorial_approach/" + n + ".csv")
#beta
table = []
for t in t_index:
    row = []
    for l in f_index:
        row.append(model.beta[t, l].value)
    table.append(row)
f_var["F"] = pd.DataFrame.from_records(table)
f_var["F"].columns = f_index
#f_var["F"].to_csv("../output/debugging/factorial_approach/beta.csv")
a_var["F"] = pd.DataFrame(
    0, index=np.arange(F.shape[0]), columns=F.columns)



In [9]:
sigma = sigma(load_dic,gen_dic,transmission_data,abbr_list)